In [2]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [4]:



# Defining a function to get a dictonary of all NFL teams.
class Nfl_stats:
    # Created an init method that sets a default dictonary that has its keays as nfl teams and values as the symbols to those teams
    
    def __init__(self):
        self.teams_symbols_dict = {}
        # setting url we want to get the data from using requests and beautifulsoup to find nfl teams 
        url = "https://www.pro-football-reference.com/teams/"
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        teams = soup.find("table", id="teams_active")

        # finding the team names in html
        all_team_names = teams.find_all("th", {"class":"left",'data-stat': 'team_name'})

        # team_names and urls for the teams
        team_names = []
        team_urls = []

        # parse throught data
        for i in all_team_names[1:]:
            team_links = i.find("a")
            team_names.append(team_links.text.lower())
            team_urls.append(team_links.get("href").split("/")[2])
        # updating teams_symbols_dict to show teams and symbols for those teams
        self.teams_symbols_dict.update(dict(zip(team_names, team_urls)))
        
            
        
    # Creating a dataframe that shows a team's stats for the season
    
    def team_stats(self, team, year):
        
        # using the teams and symbols dictonary to find the each teams data
        teams_dict = self.teams_symbols_dict
        url = "https://www.pro-football-reference.com/teams/"

        #making it easier to find the team's name in the teams_symbols_dict even if its uppercase or just the teams name not the city.
        full_team_name = ""
        for correct_team in teams_dict:
            if team in correct_team:
                full_team_name = correct_team
        
        # finding the team's stats 
        team_symbol = teams_dict[full_team_name.lower()]
        url = f"{url}{team_symbol}/{year}.htm"
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")
        table = soup.find("table", id="games")

        table_rows = table.find_all("tr")[2:]
        data = []
        for tr in table_rows:
            data.append([table_data.get_text(strip=True) for table_data in tr.find_all("td")])

        # creating a dataframe with the team's stats
        df = pd.DataFrame(data)
        
        # only getting the data from the columns that are relevent to the teams stats
        index = [0,1,4,6,8,9,10] + list(range(11,21))
        new_data = df.iloc[:,index].copy()

        # finding column names
        columns_html = table.find_all("th", {"scope":"col"})
        columns = []

        for col in columns_html[1:]:
            columns.append(col.text)
        # changing some column names and some that are duplicate so then we dont change both in dataframe later on
        columns[4] = "Result"
        columns[17] = "Toyd"
        columns[10] = "opp"
        new_data.columns = [columns[num] for num in index]
        
        # renaming columns in our dataframe so the stats match up
        team_stats_df = new_data.rename({
            "Rec": "Record", 
            "Opp": "Opposing_team",
            "Tm":"Team_score",
            "TotYd": "TO_offense",
            "Toyd": "TO_defense",
            "opp": "Opp_score"


        }, axis="columns")

        # replacing some columns into numbers so then we can do better analysis
        result_encoder = {'Result': {'L': 0, 'T': 0,'W': 1,'' : pd.NA},
                          'TO_offense' : {'' : 0},
                          'TO_defense' : {'' : 0}}
        team_stats_df.replace(result_encoder, inplace=True)
        
        # creating a week column so we know what week it is in the season
        week = list(range(1,len(team_stats_df) + 1))
        team_stats_df.insert(0, "Week", week)

        # creting a column for the team we wanted to see and making a title
        team_name_col = pd.Series([f"{team.title()}"]).repeat(len(team_stats_df)).reset_index(drop=True)

        team_stats_df.insert(0,"Team_name", team_name_col)

        return team_stats_df



    
    
    
Nfl_stats().team_stats("panthers", 2022)

,Team_name,Week,Day,Date,Result,Record,Opposing_team,Team_score,Opp_score,1stD,TO_offense,PassY,RushY,TO,1stD,TO_defense,PassY,RushY,TO
0,Panthers,1,Sun,September 11,0,0-1,Cleveland Browns,24,26,15,261,207,54,1,23,355,138,217,
1,Panthers,2,Sun,September 18,0,0-2,New York Giants,16,19,18,275,129,146,2,18,265,162,103,
2,Panthers,3,Sun,September 25,1,1-2,New Orleans Saints,22,14,12,293,148,145,,19,426,342,84,3
3,Panthers,4,Sun,October 2,0,1-3,Arizona Cardinals,16,26,11,220,180,40,3,20,338,206,132,1
4,Panthers,5,Sun,October 9,0,1-4,San Francisco 49ers,15,37,15,308,244,64,1,22,397,244,153,1
5,Panthers,6,Sun,October 16,0,1-5,Los Angeles Rams,10,24,8,203,110,93,1,22,360,249,111,1
6,Panthers,7,Sun,October 23,1,2-5,Tampa Bay Buccaneers,21,3,14,343,170,173,,17,322,276,46,
7,Panthers,8,Sun,October 30,0,2-6,Atlanta Falcons,34,37,23,478,309,169,1,21,406,239,167,2
8,Panthers,9,Sun,November 6,0,2-7,Cincinnati Bengals,21,42,13,228,164,64,3,30,464,223,241,
9,Panthers,10,Thu,November 10,1,3-7,Atlanta Falcons,25,15,18,333,101,232,,20,291,153,138,1
